In [1]:
import pandas as pd

In [2]:
class Center:
    def __init__(self,name,capacity):
        self.name = name #name of the center
        self.capacity = capacity #capacity of each center
        self.occupants = [] #an empty list of candidates to start with
        
    def add_occupant(self,student_rank):
        # student rank is the unique identifier
        # return (student added, student removed)
        if self.capacity > 0:# if there is place left, keep the student
            self.occupants.append(student_rank)# add the student
            self.occupants.sort()#keep it sorted according to rank
            self.capacity = self.capacity - 1 # reduce the capacity for intake
            return (student_rank, None)
        elif self.occupants[-1] > student_rank:# sorts by student rank, can be used different scoring here
            student_removed = self.occupants.pop()# remove the least prefered student
            self.occupants.append(student_rank) # add the new student
            self.occupants.sort() # keep it sorted ...
            return (student_rank, student_removed)
        else:
            return (None,None)

In [3]:
def allocate_centers(preference_list, centers):
    # Follows National Resident Matching Problem
    # prefrence_list is a nested list, each element is a ordered list of center names
    # centers is a dict where the keys are the center names
    # we assume number of seats are more than number of students
    num_students = len(preference_list)
    allocation_flag = [False for _ in range(num_students)] # students not allocated, rank-wise
    allocated_center = [False for _ in range(num_students)] # to be filled by the center allocated
    
    while not all(allocation_flag):# all students are required to be allocated
        for i in range(num_students):
            # trying to avoid too many indentations
            if not allocation_flag[i] and len(preference_list[i])>0 : # if not allocated or not proposed to all the centers
                center_name = preference_list[i][0]
                center = centers[center_name]
                (student_added,student_removed) = center.add_occupant(i)
                if student_added is i:
                    allocation_flag[i] = True
                    allocated_center[i] = center_name
                if student_removed is not None:
                    allocation_flag[student_removed] = False
                preference_list[i].pop(0)
            elif not allocation_flag[i] and len(preference_list[i]) == 0: # if not allocated and out of options
                allocated_center[i] = 'No Match Found.'
                allocation_flag[i] = True
                
    return allocated_center


In [25]:
# data specific utility functions

def isnan(s):
    return s != s

def read_responses(filename, all_centers):
    # read responses from students, requires recorded data in csv file
    df = pd.read_csv(filename)
    num_students = df.shape[0] # total number responses in the list
    num_centers = len(all_centers) # total number of centers
    
    preferences = {}#dictionary of preferences
    student_info = {}#dictionary of student info
    
    for i in range(num_students):
        name = df['Name'][i]
        sc = df['Student Code(Last Three Digits)'][i]
        
        if not isnan(name) and not isnan(sc): # check if the name and sc code is submitted propery
            # get the rank of the student
            rank = df['Rank'][i]
            # define response value for sorting
            def preference_value(center_name):
                return df['Preference order ['+center_name+']'].get(i,num_centers+1) 
            # add preference list accordingly
            preferences[rank] = sorted(all_centers,key = preference_value)
            student_info[rank] = {
                'Name': name,
                'ID': sc,
                'Rank': rank,
                'Center': ''
            }
    
    return preferences,student_info

def match_seats(preferences,student_info):
    # student_info is updated with the matched seats
    
    # make a nested list of preferences
    preference_list = []
    for i in sorted(preferences.keys()):
        preference_list.append(preferences[i])

    # make a dictionary of centers
    centers = {}
    for i in range(len(all_centers)):
        centers[all_centers[i]] = Center(all_centers[i],seats[i])

    allocated_centers = allocate_centers(preference_list, centers)
    count = 0
    for i in sorted(preferences.keys()):
        student_info[i]['Center'] = allocated_centers[count]
        count += 1
    return student_info

def print_match(student_info):
    # print result and return a sorted dictionary
    student_info_sorted = {}
    print('********************MOCK Allocation Results********************')
    for key in sorted(student_info.keys()):
        student_info_sorted[key] = student_info[key].copy()
        print('Rank: '+ str(student_info[key]["Rank"]) + ' | '+student_info[key]["Name"] + " SC15B" + student_info[key]["ID"] + "-->" + student_info[key]["Center"])
    return student_info_sorted

In [27]:
# To be modified accordingly

all_centers = ['IPRC','ISTRAC','NRSC','LPSC','MCF','PRL','SAC','SCL','SDSC','URSC','VSSC']
seats = [1,6,2,1,1,2,3,2,5,5,5]
preferences, student_info= read_responses('Responses.csv',all_centers)

student_info = match_seats(preferences,student_info)

student_info_sorted = print_match(student_info)

df_out = pd.DataFrame(student_info_sorted.values())
with open('Allocations.csv','w') as f: # saved in Allocations.csv file
    df_out.to_csv(f)

********************MOCK Allocation Results********************
Rank: 2 | Debajyoti Chakrabarti SC15B84-->URSC
Rank: 3 | Srinika Selvam SC15B126-->URSC
Rank: 4 | Anuj SC15B105-->URSC
Rank: 5 | Archana C M SC15B78-->VSSC
Rank: 6 | himanshu SC15B91-->SAC
Rank: 7 | Sneha Gem Mathew  SC15B124-->VSSC
Rank: 8 | Chinmay S Mhatre SC15B83-->URSC
Rank: 9 | Swetha SC15B125-->URSC
Rank: 10 | Kaninika SC15B90-->SAC
Rank: 11 | Reuben SC15B116-->VSSC
Rank: 12 | Kaustubh SC15B100-->PRL
Rank: 13 | Pragyaa SC15B107-->SAC
Rank: 14 | Rohit SC15B88-->NRSC
Rank: 15 | Suraj R SC15B129-->VSSC
Rank: 16 | Ansuman Palo SC15B76-->ISTRAC
Rank: 17 | Samvram Sahu SC15B132-->NRSC
Rank: 18 | Arnab SC15B79-->VSSC
Rank: 19 | Susmitha SC15B127-->LPSC
Rank: 20 | Sagnik SC15B117-->ISTRAC
Rank: 21 | Shubham Chauhan SC15B122-->IPRC
Rank: 22 | Ujjwala SC15B82-->ISTRAC
Rank: 23 | Sahal Mohammed M. N. SC15B118-->ISTRAC
Rank: 24 | Divyang Arora SC15B86-->ISTRAC
Rank: 25 | Mohit kumar soni SC15B103-->PRL
Rank: 26 | Raju Joarder S